In [1]:
import pandas as pd
import numpy as np  # also can use k nearest neighbour  # used in retail and e-commerce industry

In [2]:
import os
os.chdir("C:\\Users\\SHAILESH TIWARI\\Python Files\\DataSources")

In [3]:
df = pd.read_csv('Recommend.csv',names=['user_id', 'movie_id', 'rating', 'timestamp'])
df

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [4]:
from sklearn.model_selection import train_test_split
n_users = df.user_id.unique().shape[0] 
n_movies = df.movie_id.unique().shape[0]
train_data, test_data = train_test_split(df, test_size=0.25)

In [5]:
train_data_matrix = np.zeros((n_users, n_movies))
for line in train_data.itertuples():
    #[user_id index, movie_id index] = given rating.
    train_data_matrix[line[1]-1, line[2]-1] = line[3] 
train_data_matrix

array([[5., 3., 0., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [6]:
test_data_matrix = np.zeros((n_users, n_movies))
for line in test_data.itertuples():
    #[user_id index, movie_id index] = given rating.
    test_data_matrix[line[1]-1, line[2]-1] = line[3]
test_data_matrix

array([[0., 0., 4., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [7]:
from sklearn.metrics import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
movie_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')
mean_user_rating = train_data_matrix.mean(axis=1)[:, np.newaxis] 
ratings_diff = (train_data_matrix - mean_user_rating) 
user_pred = mean_user_rating + user_similarity.dot(ratings_diff) / np.array([np.abs(user_similarity).sum(axis=1)]).T
user_pred

array([[ 1.67706614,  0.58437014,  0.48788236, ...,  0.30547937,
         0.30783822,  0.30769893],
       [ 1.42540228,  0.29822713,  0.15178838, ..., -0.05601725,
        -0.05282545, -0.05255498],
       [ 1.4396467 ,  0.25435658,  0.11633266, ..., -0.09674975,
        -0.09357735, -0.09334638],
       ...,
       [ 1.28692826,  0.21614181,  0.07792248, ..., -0.12162714,
        -0.11875328, -0.11829945],
       [ 1.43825426,  0.29186148,  0.17571006, ..., -0.03123086,
        -0.02831888, -0.02803442],
       [ 1.51734964,  0.38174628,  0.28797323, ...,  0.10425774,
         0.10663195,  0.10665472]])

In [8]:
movie_pred = train_data_matrix.dot(movie_similarity) / np.array([np.abs(movie_similarity).sum(axis=1)])
movie_pred

array([[0.36791201, 0.38857683, 0.41282935, ..., 0.45627603, 0.44838349,
        0.44503691],
       [0.09534284, 0.11103969, 0.1064225 , ..., 0.11302796, 0.11267966,
        0.11350328],
       [0.07037217, 0.07428617, 0.07198311, ..., 0.07317073, 0.07279152,
        0.07387164],
       ...,
       [0.03091394, 0.04005573, 0.03901187, ..., 0.04521118, 0.04374096,
        0.04518671],
       [0.10812039, 0.11686044, 0.12473473, ..., 0.1302796 , 0.1285152 ,
        0.13021578],
       [0.20874432, 0.2013217 , 0.2263784 , ..., 0.25758477, 0.250627  ,
        0.25054822]])

In [9]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(pred, test):
    pred = pred[test.nonzero()].flatten() 
    test = test[test.nonzero()].flatten()
    return sqrt(mean_squared_error(pred, test))

In [10]:
rmse(user_pred, test_data_matrix)

3.125218415474716

In [11]:
rmse(movie_pred, test_data_matrix)

3.451026815848479